# Compound

> Fill in a module description here

In [ ]:
#| default_exp compound.core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
from dataclasses import dataclass

import chemlib
import pandas as pd
from fastcore.test import test_eq
from fastcore.basics import basic_repr

from exex.core import *
from exex.utils import *

### States

In [ ]:
#| export
@dataclass
class State:
    SOLID = 'solid'
    LIQUID = 'liquid'
    GAS = 'gas'

### Matter

In [ ]:
#| export
class Matter:
    def __init__(self):
        self.properties = dict()
        self.laws = dict()
        self.time: int = None
        self.system = None
    
    def set_time(
        self,
        time: int # time
    ):
        self.time = time
        return self
    
    def _config_laws(self, laws: list[Law]) -> None:
        for law in laws:
            name = camel_to_snake(law.__name__)
            if not name in self.laws:
                law = law(compound=self)
                law._run_config()
                self.laws[name] = law
    
    def _config(self):
        pass

All properties that a compound has always being governed by some laws.

#### Laws

In [ ]:
#| export
class MassMoleRatio(Law):
    def __init__(self, compound):
        super().__init__()
        self.compound = compound
        #self.properties = [Mass, Mole]
        self.properties = [
            {"object": Mass},
            {"object": Mole},
        ]

### Laws

In [ ]:
#| export
class Compound(Matter):
    def __init__(
        self,
        formula: str # the chemical formula
    ) -> None:
        super().__init__()
        
        compound = chemlib.Compound(formula)
        self.elements = compound.elements
        self.formula = compound.formula
        self.coefficient = compound.coefficient
        self.occurences = compound.occurences
        
        self._config_laws([MassMoleRatio])
    
    def info(self, **kwargs):
        dta = {}
        
        for k, v in self.properties.items():
            # data_point = {}
            # print(v._data)
            key = k
            # if v.unit:
            #     key += f' ({v.unit})'
        
            dta[key] = v._data
        
        df = pd.DataFrame(data=dta, **kwargs)
        df.index.name = "Time"
        return df.sort_index()
    
    def get_data(
        self,
        time: int, # the time
        name: str # the property name
    ):
        if not name in self.properties:
            return "The property don't exist"
        pass

    __repr__ = basic_repr('formula')

In [ ]:
H2O = Compound('H2O')

In [ ]:
H2O.__dict__

{'properties': {'mass': <exex.core.Mass>,
  'mole': <exex.core.Mole>},
 'laws': {'mass_mole_ratio': <__main__.MassMoleRatio>},
 'time': None,
 'elements': [<chemlib.chemistry.Element>,
  <chemlib.chemistry.Element>,
  <chemlib.chemistry.Element>],
 'formula': 'H₂O₁',
 'coefficient': 1,
 'occurences': {'H': 2, 'O': 1}}

In [ ]:
H2O.properties['mass']._data

{}

In [ ]:
#| hide
test_eq(H2O.occurences['H'], 2)
test_eq(len(H2O.elements), 3)